In [64]:
import matplotlib.pyplot as plt  
import numpy as np
import math
import scipy.io as sio
from andylearn import mnist, KNNneighbor
import pylab
#%matplotlib inline

In [65]:
def L2_Graph_Learning(data, k, d, lamb=1):
    data = np.mat(data).T
    n_features, n_samples = data.shape
    C = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        X = data.copy()
        X = np.delete(X, i, axis=1)
        X = np.insert(X, i, np.zeros((1, n_features)), axis=1)
        # L2-graph
        P = np.dot(X.T, X)+ lamb*np.eye(n_samples)
        P = P.I
        Q = np.dot(P, X.T)
        ei = np.zeros((n_samples-1, 1))
        ei = np.insert(ei, i, 1)
        xi = data[:, i]

        term = np.dot(X.T, xi) 
        f1 = float(ei.T.dot(Q).dot(xi)) * ei
        f2 = float(np.mat(ei.T.dot(P).dot(ei)))
        term = term - (f1/f2).reshape(n_samples, 1)
        ci = np.dot(P, term).getA1()
        ci_sort = np.argsort(ci)
        ci[ci_sort[0:n_samples-k]] = 0 
        C[i] = ci
        
    W = np.abs(C) + np.abs(C.T)
    for i in range(W.shape[1]):
        W[:, i] = np.divide(W[:, i], np.linalg.norm(W[:, i]))
    M = np.eye(n_samples)-W
    M = np.dot(M, M.T)
    
    Lterm = np.dot(np.dot(data, M), data.T)
    Rterm = np.dot(data, data.T)
    tol = 1e-6
    Rterm = Rterm + np.eye(Rterm.shape[0])*tol
    eigenvalues, eigenvectors = np.linalg.eig(np.dot(Rterm.I, Lterm))
    
    eigen_sort = np.argsort(eigenvalues)
    k_eigenvectors = eigenvectors[:, eigen_sort[0:d]]
    return k_eigenvectors

In [68]:
def L2_ORL_acuracy(l, k, d):
    ## 32 * 32 ORL_faces 
    s=0.0
    for i in range(50):
        matfn = 'data/orl_faces/32_32/'+str(l)+'Train/'+str(i+1)+'.mat'
        index = sio.loadmat(matfn)
        testIdx = index['testIdx'] -1  
        trainIdx = index['trainIdx'] -1
        trainface = face[trainIdx]
        trainlabel = np.squeeze(label[trainIdx])
        trainface = np.squeeze(trainface) 
    
        
        testface = face[testIdx]
        testlabel =  np.squeeze(label[testIdx])
        testface = np.squeeze(testface)  
       
        mapping = L2_Graph_Learning(trainface, k, d).T
        
        traindata = np.dot(mapping, trainface.T)
        testdata = np.dot(mapping, testface.T)
        tmp = KNNneighbor.accuracy(traindata.T, trainlabel, testdata.T, testlabel, 1)
        print(tmp)
        s += tmp
        
    return s/50

print(L2_ORL_acuracy(5, 100, 100))

/Users/killandy/Library/Python/3.6/lib/python/site-packages/numpy/core/numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


0.045
0.045
0.02
0.055
0.07
0.06
0.05
0.08
0.02
0.025
0.03
0.08
0.025
0.025
0.03
0.045
0.025
0.025
0.025
0.03
0.02
0.02
0.04
0.04
0.05
0.02
0.02
0.03
0.02
0.03
0.03
0.04
0.04
0.02
0.035
0.045
0.02
0.015
0.045
0.035
0.02
0.01
0.015
0.025
0.03
0.025
0.035
0.025
0.03
0.02
0.0332
